# Temporary Worksapce

To test things out

In [1]:
using Muon
using RData
using Random
using DataFrames

In [2]:
adata = readh5ad("../data/SCT.h5ad")
lr_network = load("../data/lr_network.rds");

In [3]:
display(first(adata.obs, 5))
display(first(adata.var, 5))
adata.obsm

Row,CompositionCluster_CC,predicted.id,nCount_Spatial,nFeature_Spatial,nCount_SCT,nFeature_SCT,Deconvoluted_B_Prob,Deconvoluted_CD4.T_Prob,Deconvoluted_CD8..T_Prob,Deconvoluted_DC_Prob,Deconvoluted_Endothelial_Prob,Deconvoluted_Enterocytes_Prob,Deconvoluted_Fibroblasts_Prob,Deconvoluted_Goblet_Prob,Deconvoluted_ILCs_Prob,Deconvoluted_Inflammatory.Fibroblasts_Prob,Deconvoluted_Macrophages_Prob,Deconvoluted_Mast_Prob,Deconvoluted_Mcells_Prob,Deconvoluted_Monocytes_Prob,Deconvoluted_Neutrophils_Prob,Deconvoluted_NK_Prob,Deconvoluted_Plasma_Prob,Deconvoluted_TA_Prob,Deconvoluted_Tregs_Prob,Deconvoluted_Tuft_Prob
,Cat…,Cat…,Float64,Int32,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CC6,TA,9500.0,3382,5091.0,2767,0.110827,0.0362491,0.0629087,0.0,0.00409948,0.0466145,0.00642297,0.087586,0.0,0.00906009,0.0,0.025285,0.00689851,0.0517618,0.0464921,0.0,0.0362959,0.469499,0.0,0.0
2,CC7,Plasma,13031.0,4195,4505.0,2264,9.07313e-18,0.000244837,0.0567895,0.00208808,0.0173666,0.0454543,0.0112047,0.015053,0.0132461,0.0214567,0.320661,1.54369e-17,0.00123446,0.00964091,0.0583849,0.0,0.0663191,0.262881,0.0453937,0.0525808
3,CC8,CD4,4198.0,2591,4187.0,2580,0.182994,0.140278,0.233147,0.0104887,0.0668716,0.0,0.0199931,0.00414359,0.0309342,0.0351498,0.0770044,0.0222782,0.0014616,0.0174124,0.0254909,0.0,0.0,0.0115749,0.120778,0.0
4,CC3,Fibroblasts,1105.0,751,3221.0,1004,0.162122,0.0352555,0.0504496,0.0,0.0233267,0.0,0.0212781,0.0900401,0.00561752,0.0986164,0.054518,5.59361e-18,6.38478e-5,0.0335658,0.0332916,0.0,0.0262774,0.365578,0.0,0.0
5,CC7,Postcapillary_Venules,8727.0,3517,5303.0,3109,0.0358204,0.0209851,0.030029,0.0,0.153044,0.00529109,0.0222282,0.040053,0.0255154,0.114603,0.0986734,0.0443718,0.00757579,0.0489262,0.115696,0.0,0.0342289,0.194998,0.0,0.00796098


Row,name
,String
1,AL627309.1
2,AL627309.5
3,LINC01409
4,LINC01128
5,LINC00115


Muon.AlignedMapping{Tuple{1 => 1}, String, AnnData} with 2 entries:
  "X_pca"  => [4.01475 11.0479 … -0.390483 0.810208; -3.26189 -3.26585 … 1.5099…
  "X_umap" => [1.10153 3.72744; 8.36962 1.93914; … ; 6.32683 4.19993; 5.8863 4.…

In [12]:
# Create LR_Pairs column
lr_network[!, :LR_Pairs] = string.(lr_network.from, "_", lr_network.to);
first(lr_network, 5)

Row,from,to,source,database,LR_Pairs
,String,String,String,String,String
1,CXCL1,CXCR2,kegg_cytokines,kegg,CXCL1_CXCR2
2,CXCL2,CXCR2,kegg_cytokines,kegg,CXCL2_CXCR2
3,CXCL3,CXCR2,kegg_cytokines,kegg,CXCL3_CXCR2
4,CXCL5,CXCR2,kegg_cytokines,kegg,CXCL5_CXCR2
5,PPBP,CXCR2,kegg_cytokines,kegg,PPBP_CXCR2
